# TFM
# Modelo LSTM

REALIZADO POR: JOSE XAVIER MONAR MEJIA <br>
FECHA: ENERO/2025

# **Preparación del Entorno para el Modelo LSTM**

En este bloque, importaremos todas las librerías necesarias para implementar un modelo LSTM. Estas incluyen herramientas para la manipulación de datos, visualización, preprocesamiento, construcción del modelo, entrenamiento y evaluación. A continuación, detallamos el propósito de cada librería:

- **Manipulación de datos**:
  - `pandas`: Para cargar y procesar datos estructurados.
  - `numpy`: Para operaciones matemáticas y manejo de arrays.

- **Visualización de datos**:
  - `matplotlib`: Para generar gráficos básicos.
  - `seaborn`: Para gráficos más estilizados y análisis exploratorios.

- **Preprocesamiento y escalado**:
  - `sklearn.preprocessing.MinMaxScaler`: Para normalizar los datos entre 0 y 1.

- **Construcción y entrenamiento del modelo**:
  - `tensorflow.keras`: Para construir, entrenar y evaluar el modelo LSTM.
    - `Sequential`: Para estructurar el modelo en capas.
    - `LSTM`: Para añadir capas LSTM al modelo.
    - `Dense`: Para añadir capas completamente conectadas.
    - `Dropout`: Para evitar el sobreajuste.

- **Evaluación del modelo**:
  - `sklearn.metrics`: Para calcular métricas de rendimiento como MAE, MSE y RMSE.

- **Configuración del entorno**:
  - `os`: Para gestionar rutas de archivos si es necesario.
  - `warnings`: Para suprimir advertencias innecesarias.

A continuación, realizamos las importaciones correspondientes.


In [1]:
import tensorflow as tf
print(f"TensorFlow instalado correctamente, versión: {tf.__version__}")

TensorFlow instalado correctamente, versión: 2.18.0


In [2]:
# Librerías para manipulación de datos
import pandas as pd  # Manejo de estructuras de datos como DataFrames
import numpy as np   # Operaciones matemáticas y arrays

# Librerías para visualización
import matplotlib.pyplot as plt  # Gráficos básicos
import seaborn as sns  # Visualizaciones avanzadas

# Preprocesamiento de datos
from sklearn.preprocessing import MinMaxScaler  # Normalización de datos

# Construcción y entrenamiento del modelo LSTM
from tensorflow.keras.models import Sequential  # Para crear el modelo
from tensorflow.keras.layers import LSTM, Dense, Dropout  # Capas LSTM y Dense
from tensorflow.keras.optimizers import Adam  # Optimizador

# Evaluación del modelo
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Configuración del entorno
import os  # Manejo de rutas
import warnings  # Supresión de advertencias
warnings.filterwarnings('ignore')  # Ignorar advertencias innecesarias

# Configuración de estilo para los gráficos
plt.style.use('ggplot')

# Confirmación de que las librerías han sido cargadas correctamente
print("Librerías importadas con éxito.")

Librerías importadas con éxito.


# **Carga del Dataset**

En este bloque, cargaremos el dataset ubicado en la siguiente URL:
`https://github.com/huambra/Predicting-Shrimp-Exports/raw/refs/heads/main/Exports%20por%20Mercado.xlsx`.

## Pasos:
1. Importaremos la librería `pandas`, que nos permite leer archivos Excel.
2. Usaremos la función `read_excel` para cargar el archivo.
3. Mostraremos las primeras filas del dataset para asegurarnos de que se cargó correctamente.


In [4]:
# Ruta del archivo
file_path = "https://github.com/huambra/Predicting-Shrimp-Exports/raw/refs/heads/main/Exports%20por%20Mercado.xlsx"

# Cargamos el dataset
try:
    data = pd.read_excel(file_path, sheet_name="Unpivot")
    # Mostramos las primeras filas del dataset
    print("Primeras filas del dataset:")
    print(data.head())
    
    # Verificamos la estructura del dataset
    print("\nInformación del dataset:")
    print(data.info())
except Exception as e:
    print(f"Error al cargar el dataset: {e}")


Primeras filas del dataset:
        Date  China   USA    UE
0 2012-12-01   19.0   291    75
1 2013-01-01  740.0  4119  4241
2 2013-02-01  380.0  5844  5407
3 2013-03-01  459.0  6929  7510
4 2013-04-01  403.0  7266  7830

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    141 non-null    datetime64[ns]
 1   China   141 non-null    float64       
 2   USA     141 non-null    int64         
 3   UE      141 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 4.5 KB
None
